# Парсинг Elibrary

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
from tqdm.notebook import tqdm
import nltk
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

Предварительно необходимо создать подборки по преподавателям на Elibrary

In [ ]:
dict_of_teachers = {'Вилькен_Виктория_Валерьевна' : 2, 'Ливинцова_Мария_Геннадьевна' : 3,
                    'Климин_Анастасий_Игоревич' : 4, 'Лукашевич_Никита_Сергеевич' : 5,
                    'Абушова_Екатерина_Евгеньевна' : 6, 'Кальченко_Ольга_Александровна' : 7,
                    'Тихонов_Дмитрий_Владимирович' : 8, 'Дуболазов_Анатолий_Александрович' : 9,
                    'Симакова_Зоя_Леонидовна' : 10, 'Авдуевская_Екатерина_Алексеевна' : 11,
                    'Богданова_Татьяна_Александровна' : 12, 'Фадеев_Алексей_Михайлович' : 13,
                    'Бабкин_Иван_Александрович' : 14, 'Пупенцова_Светлана_Валентиновна' : 15,
                    'Гутман_Светлана_Семеновна' : 16, 'Дмитриев_Николай_Дмитриевич' : 17,
                    'Дуболазова_Юлия_Андреевна' : 18, 'Кожина_Ксения_Сергеевна' : 19,
                    'Конников_Евгений_Александрович' : 20, 
                    'Муханова_Наталья_Викторовна' : 22, 'Некрасова_Татьяна_Петровна' : 23,
                    'Пашоликов_Максим_Александрович' : 24, 'Покровская_Любовь_Леонидовна' : 25,
                    'Рудская_Ирина_Андреевна' : 26, 'Схведиани_Анги_Ерастиевич' : 27,
                    'Томшинская_Ирина_Николаевна' : 28, 'Левенцов_Валерий_Александрович': 29}

password = "Ваш_пароль"
login = "Ваш_логин"

options = webdriver.ChromeOptions()
# options.add_argument("--headless")

options.add_experimental_option('excludeSwitches', ['enable-automation'])

options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(executable_path='chromedriver', chrome_options = options)

driver.implicitly_wait(2)

driver.maximize_window()

driver.get('https://www.elibrary.ru/itembox_items.asp?id=1369511')
time.sleep(random.uniform(0, 2))

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td[1]/table/tbody/tr[6]/td/div/table/tbody/tr/td/table[1]/tbody/tr[6]/td/input'))).send_keys(login)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td[1]/table/tbody/tr[6]/td/div/table/tbody/tr/td/table[1]/tbody/tr[8]/td/input'))).send_keys(password)

WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td[1]/table/tbody/tr[6]/td/div/table/tbody/tr/td/table[1]/tbody/tr[9]/td/input'))).click()

for k, v in dict_of_teachers.items():
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table/tbody/tr/td[2]/table/tbody/tr[{v}]/td[2]/font/a'))).click()                                  
    time.sleep(random.uniform(1, 5))
    test = driver.page_source
    
    index_for_pages = 4
    quantity_of_back = 1

    quantity_of_articles = len(BeautifulSoup(test).findAll('span', {'style' : 'line-height:1.0;'}))

    inf_about_requests = []

    for i in tqdm(range(2, quantity_of_articles + 2)):
        try:
            try:
                try:
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[3]/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table[2]/tbody/tr[{i}]/td[2]/a/b/span'))).click()
                except:
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[2]/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table[2]/tbody/tr[{i}]/td[2]/a/b/span'))).click()
            except:
                driver.back()
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[3]/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table[2]/tbody/tr[{i}]/td[2]/a/b/span'))).click()
                WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table[1]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/table[1]/tbody/tr/td[2]/font/a')))
        except:
            driver.back()
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[3]/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table[2]/tbody/tr[{i}]/td[2]/a/b/span'))).click()
        try:
            WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/table/tbody/tr/td/table[1]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/div/table[5]/tbody/tr[2]/td[2]/div[3]/a'))).click()
        except:
            time.sleep(0.1)
        time.sleep(random.uniform(0, 2))
        WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table[1]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/table[1]/tbody/tr/td[2]/font/a')))
        inf_about_requests.append(driver.page_source)
        driver.back()
        time.sleep(random.uniform(0, 2))
        
    while quantity_of_articles == 100:
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[3]/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table[3]/tbody/tr/td[{index_for_pages}]/a'))).click()
        time.sleep(2)
        test = driver.page_source
        quantity_of_articles = len(BeautifulSoup(test).findAll('span', {'style' : 'line-height:1.0;'}))                                                           
        for i in tqdm(range(2, quantity_of_articles + 2)):
            try:
                try:
                    try:
                        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[3]/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table[2]/tbody/tr[{i}]/td[2]/a/b/span'))).click()
                    except:
                        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[2]/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table[2]/tbody/tr[{i}]/td[2]/a/b/span'))).click()
                except:
                    driver.back()
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[3]/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table[2]/tbody/tr[{i}]/td[2]/a/b/span'))).click()
                    WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table[1]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/table[1]/tbody/tr/td[2]/font/a')))
            except:
                driver.back()
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[3]/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr[3]/td/form/table[2]/tbody/tr[{i}]/td[2]/a/b/span'))).click()
            try:
                WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/table/tbody/tr/td/table[1]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/div/table[5]/tbody/tr[2]/td[2]/div[3]/a'))).click()
            except:
                time.sleep(0.1)
            time.sleep(random.uniform(0, 2))
            WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table[1]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/table[1]/tbody/tr/td[2]/font/a')))
            inf_about_requests.append(driver.page_source)
            driver.back()
            time.sleep(random.uniform(0, 2))
        index_for_pages += 1
        quantity_of_back += 1
    
    TITLE = []
    AUTHORS = []
    ABSTRACT = []
    UNIVETSITY = []
    SOURCE = []

    for x in tqdm(inf_about_requests):
        TITLE.append(BeautifulSoup(x).find('p', {'class' : 'bigtext'}).text)
        oz_1 = []
        oz = []
        m = BeautifulSoup(x).findAll('span', {'class' : 'help pointer'})
        for i in m:
            if '1' in i.text:
                new = re.sub('1', '', i.text)
                oz.append(new)
            else:
                oz.append(i.text)
        authors = ' '.join(oz)
        AUTHORS.append(authors)
        try:
            ABSTRACT.append(BeautifulSoup(x).find('p', {'align' : 'justify'}).text)
        except:
            ABSTRACT.append('-')
        university_1 = []
        m = BeautifulSoup(x).findAll('span', {'class' : 'help1 pointer'})
        for i in m:
            university_1.append(i.text)
        new = ' '.join(university_1)
        UNIVETSITY.append(new)
        m = BeautifulSoup(x).findAll('td', {'width' : '504'})
        oz = []
        for i in m:
            oz.append(i.text)
            source = ' '.join(oz)
        SOURCE.append(source)

    print(len(TITLE))
    print(len(AUTHORS))
    print(len(ABSTRACT))
    print(len(UNIVETSITY))
    print(len(SOURCE))

    full_list = list(zip(TITLE, AUTHORS, ABSTRACT, UNIVETSITY, SOURCE))
    df = pd.DataFrame(full_list, columns= ['Название статьи', 'Авторы', 'Аннотация', 'Организация', 'Источник'])
    df.to_excel(f'{k}.xlsx')
    for i in range(quantity_of_back):
        driver.back()

Фильтрация собранных данных

In [ ]:
substrings = ['Санкт-Петербургский политехнический',
              'Peter the Great St',
              'St. Petersburg Polytechnic',
              'St. Petersburg State',
              'СПбГПУ',
              'Санкт-Петербургский государственный политехнический',
              'Saint-Petersburg State Polytechnical University',
              'Peter the Great Saint',
              'СПбПУ', 'Институт экономических проблем',
              'Институт промышленного менеджмента', 'Institute of industrial management']

substrings_1 = ['НЕДЕЛЯ НАУКИ СПБПУ', 'Издательство Политехнического университета',
                'Санкт-Петербургский политехнический университет Петра Великого', 'ПОЛИТЕХ-ПРЕСС']

regex_pattern = '|'.join(map(re.escape, substrings))
regex_pattern_1 = '|'.join(map(re.escape, substrings_1))

for sci in tqdm(list(dict_of_teachers.keys())):
    df = pd.read_excel(f'{sci}.xlsx')
    filt = (df['Организация'].str.contains(regex_pattern, case=False) | df['Источник'].str.contains(regex_pattern_1, case=False))
    df[filt].to_excel(f'{sci}.xlsx')

# Парсинг групп ВК

In [ ]:
import requests 
from tqdm.notebook import tqdm 
import time
import pandas as pd

Предварительно необходимо узнать id страниц

In [ ]:
dict_of_teachers = {'Абушова_Екатерина_Евгеньевна' : '213551',
                   'Авдуевская_Екатерина_Алексеевна' : '87209380',
                   'Бабкин_Иван_Александрович' : '93936875',
                   'Богданова_Татьяна_Александровна' : '1934831',
                   'Вилькен_Виктория_Валерьевна' : '27427904',
                   'Гутман_Светлана_Семеновна' : '1492436',
                   'Дмитриев_Николай_Дмитриевич' : '79214648',
                   'Дуболазов_Анатолий_Александрович' : '289926',
                   'Дуболазова_Юлия_Андреевна' : '17902599',
                   'Кальченко_Ольга_Александровна' : '49222',
                   'Климин_Анастасий_Игоревич' : '3396625',
                   'Кожина_Ксения_Сергеевна' : '132657764',
                   'Конников_Евгений_Александрович' : '1083727',
                   'Левенцов_Валерий_Александрович' : '152326',
                   'Ливинцова_Мария_Геннадьевна' : '1029537',
                   'Лукашевич_Никита_Сергеевич' : '35147',
                   'Муханова_Наталья_Викторовна' : '4210429',
                   'Некрасова_Татьяна_Петровна' : '1425316',
                   'Пашоликов_Максим_Александрович' : '5648',
                   'Покровская_Любовь_Леонидовна' : '8542953',
                   'Пупенцова_Светлана_Валентиновна' : '1885157',
                   'Рудская_Ирина_Андреевна' : '89418228',
                   'Симакова_Зоя_Леонидовна' : '1652858',
                   'Схведиани_Анги_Ерастиевич' : '31369407',
                   'Тихонов_Дмитрий_Владимирович' : '4550931',
                   'Томшинская_Ирина_Николаевна' : '37281427',
                   'Фадеев_Алексей_Михайлович' : '18914447'}

url = 'https://api.vk.com/method/groups.get'

params = {'access_token':'Ваш_ключ_доступа', 
          'v': '5.131',
          'extended' : '1',
          'fields': 'description'}

for k, v in tqdm(dict_of_teachers.items()):
    params['user_id'] = v
    inf = requests.get(url, params = params).json()
    description = []
    for group in range(len(inf['response']['items'])):
        try:
            description.append(inf['response']['items'][group]['description'])
        except:
            description.append('-')
    df = pd.DataFrame(description, columns = ['Описание групп'])
    df.to_excel(f'Описание_групп_{k}.xlsx', index=False)
    

# Векторизация и целевой коэффициент сходства

In [ ]:
import nltk
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

In [ ]:
list_of_vectors = []
morf = pymorphy2.MorphAnalyzer()

for sci in tqdm(list(dict_of_teachers.keys())):
    df = pd.read_excel(f'{sci}.xlsx')
    texts = list(df['Название статьи']) + list(df['Аннотация']) 
    tokkens_lists = []
    for sentence in texts:
        tokkens_lists.extend(nltk.word_tokenize(sentence))
    sent_teg_list = ['NOUN', 'ADJF', 'ADJS', 'VERB', 'INFN', 'ADVB']
    tokens_1 = []
    for token in tokkens_lists:
        token_morf_inf = morf.parse(token)
        if token_morf_inf[0].tag.POS in sent_teg_list:
            tokens_1.append(token_morf_inf[0].normal_form)
    vector = ' '.join(tokens_1)
    list_of_vectors.append(vector)
    
tfidfconverter = TfidfVectorizer(max_features = 5000)

X = tfidfconverter.fit_transform(list_of_vectors).toarray()
    
df_vect = pd.DataFrame(X)

df_as_list = df_vect.apply(lambda row: row.tolist(), axis=1)
vectors = list(df_as_list)
vectors_array = np.array(vectors)
norms = np.linalg.norm(vectors_array, axis=1)
dot_products = np.dot(vectors_array, vectors_array.T)
cosine_similarities = dot_products / np.outer(norms, norms)

df_vect_1 = pd.DataFrame(cosine_similarities)
df_vect_1.insert(0, ' ', list(dict_of_teachers.keys()))
for i in range(len(dict_of_teachers)):
    df_vect_1 = df_vect_1.rename(columns={f'{i}': f'{list(dict_of_teachers.keys())[i]}'})
df_vect_1.columns = [[' '] + list(dict_of_teachers.keys())]

df_vect_1.to_excel('Название_эксель_файла.xlsx')

Точно такой же алгоритм для групп вк

In [ ]:
# cosine_similarities_1 - матрица косинусного сходства по группам вк
cosine_similarities_2 = cosine_similarities_1*0.5 + cosine_similarities*0.5
df_vect_new_2 = pd.DataFrame(cosine_similarities_2)
df_vect_new_2.insert(0, ' ', list(dict_of_teachers.keys()))
for i in range(len(dict_of_teachers)):
    df_vect_new_2 = df_vect_new_2.rename(columns={f'{i}': f'{list(dict_of_teachers.keys())[i]}'})
df_vect_new_2.columns = [[' '] + list(dict_of_teachers.keys())]
df_vect_new_2.to_excel('Коэфф_сходства.xlsx')